In [83]:
import numpy as np
import matplotlib.pyplot as plt
import copy
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
import os

np.random.seed(1)
torch.manual_seed(1)

# Define Transformer


# (t,rho,z) -> (t',rho',z')
class T(nn.Module):
    def __init__(self,w=256,a=0.,M=1.):
        super(T, self).__init__()
        self.l1 = nn.Linear(2,w)
        self.l2 = nn.Linear(w,w)
        self.l3 = nn.Linear(w,4)
        self.a = a
        self.M = torch.nn.Parameter(torch.ones(1,)*1., requires_grad=False)
        self.eps = torch.nn.Parameter(torch.ones(1,)*0.1, requires_grad=False)
    
    def forward(self, x):
        bs = x.shape[0]
        f = nn.ReLU()
        self.t = x[:,[0]]
        self.x = x[:,[1]]
        self.y = x[:,[2]]
        self.z = x[:,[3]]
        self.r = torch.sqrt(self.x**2+self.y**2+self.z**2)
        self.u = torch.sqrt(self.r/(2*self.M))

        self.rho = torch.unsqueeze(torch.sqrt(x[:,1]**2+x[:,2]**2),dim=1)
        self.rhoz = torch.transpose(torch.stack([self.rho,self.z]),0,1)[:,:,0]
        
        self.x1 = f(self.l1(self.rhoz))
        self.x2 = f(self.l2(self.x1))
        self.x3 = self.l3(self.x2)
        
        self.dt = self.x3[:,[0]]
        self.drho = self.x3[:,[1]]
        self.dphi = self.x3[:,[2]]
        self.dz = self.x3[:,[3]]
        nn_out = torch.empty((x.shape[0], 4), requires_grad=False)
        nn_out[:,[0]] = self.dt
        nn_out[:,[1]] = (-self.y*self.dphi+self.x*self.drho)
        nn_out[:,[2]] = (self.x*self.dphi+self.y*self.drho)
        nn_out[:,[3]] = self.dz*self.z

        return x + self.eps.unsqueeze(dim=0)*nn_out
    
    def set_a(self,a):
        self.a = a
        
    def batch_jacobian(self, func, x, create_graph=False):
        # x in shape (Batch, Length)
        def _func_sum(x):
            return func(x).sum(dim=0)
        return torch.autograd.functional.jacobian(_func_sum, x, create_graph=create_graph).permute(1,0,2)
    
    def transform_g(self, x):
        jac_ts = self.batch_jacobian(self.forward, x, create_graph=True)
        jac_inv_ts = torch.inverse(jac_ts)
        return torch.matmul(torch.matmul(jac_inv_ts.permute(0,2,1), g(x)),jac_inv_ts)
        
    def jac(self, x):
        jac_ts = self.batch_jacobian(self.forward, x, create_graph=True)
        return jac_ts
    

In [84]:
w_s = 10
w_l = 20
t1 = T(w=w_s)
t2 = T(w=w_l)

In [85]:
def grow(t1, t2, w_s, w_l):

    w_mask = torch.zeros(w_l,w_l)
    w_mask[:w_s,:w_s] = torch.ones(w_s,w_s)

    b_mask = torch.zeros(w_l,)
    b_mask[:w_s] = torch.ones(w_s,)

    t2.l2.weight.data = t2.l2.weight.data*w_mask
    t2.l2.weight.data[:w_s,:w_s] = t1.l2.weight.data
    t2.l2.bias.data = t2.l2.bias.data*b_mask
    t2.l2.bias.data[:w_s] = t1.l2.bias.data

    t2.l1.weight.data[:w_s,:] = t1.l1.weight.data
    t2.l1.bias.data[:w_s] = t1.l1.bias.data

    t2.l3.weight.data[:,:w_s] = t1.l3.weight.data
    t2.l3.bias.data[:w_s] = t1.l3.bias.data
    return t2

In [86]:
t2 = grow(t1,t2,w_s,w_l)

In [87]:
x = torch.normal(0,1,size=(100,4))

In [88]:
t2(x)-t1(x)

tensor([[ 0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00],
        [ 0.0000e+00,  0.0000e+00, -5.9605e-08,  0.0000e+00],
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00],
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00],
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00],
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00],
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00],
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00],
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00],
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00],
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00],
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00],
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00],
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00],
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00],
        [ 0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00],
        